# LLM-Based D&D Simulation Tutorial

This notebook demonstrates how to use the LLM-based D&D simulation system to create AI-driven gameplay sessions based on real human campaign data.

## Overview

The simulation system includes:
- **Campaign Parameter Extraction**: Analyze human campaigns to extract initialization parameters
- **Character Creation**: Generate D&D characters with personalities and classes
- **Turn Sampling**: Create realistic turn sequences using various sampling methods
- **Character Agents**: LLM-powered characters with memory and decision-making
- **Game Session Management**: Orchestrate complete D&D scenarios

## Setup and Imports

In [1]:
import os
import sys
import json
import random
from pathlib import Path
import textwrap
import numpy as np
import datetime
import os, sys
from collections import Counter

# Import our simulation system
import llm_scaffolding.dnd_simulation as sim
import llm_scaffolding

# Set random seed for reproducible results
#random.seed(42)


✅ Anthropic API key loaded
✅ OpenAI API key loaded
✅ Gemini API key loaded


## Model Selection

The simulation system supports multiple LLM providers through liteLLM. You can use:

- **Anthropic Claude**: `claude-3-5-sonnet-latest`, `claude-3-haiku-latest`
- **OpenAI GPT**: `gpt-4o`, `gpt-4o-mini`, `gpt-3.5-turbo`
- **Google Gemini**: `gemini-pro`, `gemini-pro-vision`

Make sure your API keys are configured in `api_key.txt` in the project root.

## Step 1: Extract Campaign Parameters

Let's load a real human campaign and extract parameters that we can use to initialize our simulation.

In [51]:
# Set parameters
campaign_name = '20049-age-of-death' #Options: '7426-serpent-isle-private-campaign'#'20049-age-of-death' #'3135-the-head-of-the-serpent' # '89221-banhaven-academy-year-1' # '10391-guardians-of-gridori' '90630-firecat5s-dragon-of-icespire-peak' '1262-firedeath-must-die-closed-play-by-post-open-ogg'
campaign_file = os.path.join(os.getcwd(),"..", "data","raw-human-games", "individual_campaigns", campaign_name + '.json')
include_player_personalities = False
scratchpad = True
cache_update_interval = 10

# You can specify different models for different providers:
# model = "claude-3-5-sonnet-20240620"  # Anthropic Claude Sonnet (default)
# model = "claude-3-haiku-20240307"     # Anthropic Claude haiku
# model = "gpt-4o", "gpt-4o-mini"       # OpenAI GPT-4
# model = "gemini-pro"                  # Google Gemini
model = "claude-3-7-sonnet-latest" #"claude-3-7-sonnet-latest"# "gemini/gemini-1.5-pro" #"gpt-4o" ##
llm_scaffolding.DEFAULT_TEMPERATURE = 1.5  # geimini and gpt: 1.5, claude = 1

# Extract parameters from the human campaign
print("📊 Extracting campaign parameters...")
campaign_params = sim.extract_campaign_parameters(campaign_file, model=model)

print(f"\n🎯 Campaign Parameters:")
print(f"  Campaign Name: {campaign_params['campaign_name']}")
print(f"  Total Messages: {campaign_params['total_messages']:,}")
print(f"  Number of Players: {campaign_params['num_players']}")
print(f"  Character Names: {campaign_params['character_names']}")
print(f"  Player Names: {campaign_params['player_names']}")
print(f"  Character Classes: {campaign_params['character_classes']}")
print(f"  Character Genders: {campaign_params['character_genders']}")
print(f"  Character Races: {campaign_params['character_races']}")
print(f"  Initial Scenario: {campaign_params['initial_scenario']}")

print('Character Personalities + Sheets')
for i in range(len(campaign_params['character_names'])):
    print(textwrap.fill(f"{campaign_params['character_names'][i]}: {campaign_params['character_personalities'][i]}", width=170))
    print('\n')
    print('Sheet: ')
    if isinstance(campaign_params['character_sheets'][i], dict):
        for key, value in campaign_params['character_sheets'][i].items():
            print(f"{key}: {value}")
    else:
        print(campaign_params['character_sheets'][i])
    print('\n')

print('Player Personalities')
for i in range(len(campaign_params['player_names'])):
    print(
        textwrap.fill(
            f"{campaign_params['player_names'][i]}: {campaign_params['player_personalities'][i]}",
            width=170))
    print('\n')


📊 Extracting campaign parameters...

🎯 Campaign Parameters:
  Campaign Name: 20049-age-of-death
  Total Messages: 186
  Number of Players: 4
  Character Names: ['Dungeon Master' 'Finlan' 'Masque' 'Cororin']
  Player Names: ['Batman1122334456', 'Tyrannosary', 'Benmasque', 'tommatherii']
  Character Classes: [None, 'barbarian', 'Cleric', None]
  Character Genders: [None, 'male', 'male', 'male']
  Character Races: [None, {}, {}, {}]
  Initial Scenario: {'1': {'date': '2025-08-18T13:15:42.844694', 'player': 'Batman1122334456', 'character': 'Dungeon Master', 'in_combat': False, 'paragraphs': {'0': {'text': 'WHAT:\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 

## Step 2: Create Characters

Now let's create our AI characters using the player count from the human campaign.

In [52]:
# Create characters based on the campaign parameters
print("🧙 Creating characters...")

characters = sim.create_characters(campaign_params, model=model)

print(f"\n✨ Created {len(characters)} characters:")
for char in characters:
    print(f"  🗡️  {char.name} ({char.dnd_class}): {char.personality}")
    print(f"         {char.character_sheet}")

# Get character names for turn sampling
character_names = [char.name for char in characters]
print(f"\n👥 Character roster: {character_names}")

🧙 Creating characters...

✨ Created 4 characters:
  🗡️  Dungeon Master (None): None
         (None,)
  🗡️  Finlan (barbarian): Finlan is a halfling barbarian who embodies the duality of small stature but ferocious spirit. Quick to demonstrate his battle prowess, he possesses a cheerful, somewhat naive personality that belies his combat skills. Finlan tends to view the world with a childlike optimism, evident in his amusing misinterpretation of dead horses as "playing dead to please them." Though he may not excel at stealth or investigation, he compensates with enthusiasm and combat readiness. Finlan displays a streak of pride about his racial identity, taking offense when overlooked and enjoying dwarven ale over traditional halfling brews. He's particularly distrustful of suspicious characters like the "goliath pervert" Cororin, keeping a watchful eye on potential troublemakers. Finlan's straightforward approach to problems and his eagerness to participate in group activities suggest h

## Step 3: Generate Turn Sequence

Create a realistic turn sequence using uniform sampling across all characters.

In [53]:
# Generate a turn sequence 
total_turns = len(campaign_params['character_turns'])-1  # Adjust for desired session length
from_human_game = True

if from_human_game:
    turn_sequence = np.array(campaign_params['character_turns'])[1:total_turns] # start at 1 to remove 1st DM post
else:
    turn_sequence = sim.sample_turn_sequence(
    character_names=character_names,
    total_turns=total_turns,
    method='uniform')

#print(f"\n📋 Turn sequence: {turn_sequence}")

# Show turn distributions
print('Total turn number: ' + str(total_turns))
turn_counts = Counter(turn_sequence)
print(f"\n📊 Turn distribution:")
for char_name, count in turn_counts.items():
    print(f"  {char_name}: {count} turns ({count/total_turns*100:.1f}%)")

Total turn number: 183

📊 Turn distribution:
  Finlan: 67 turns (36.6%)
  Masque: 20 turns (10.9%)
  Dungeon Master: 60 turns (32.8%)
  Cororin: 35 turns (19.1%)


## Step 4: Initialize Game Session

Create the game session manager that will orchestrate our D&D simulation.

In [54]:
# Create game session
print("🎭 Initializing game session...")
game_session = sim.GameSession(characters=characters,
                               campaign_name=campaign_name, 
                               cache_update_interval=cache_update_interval,
                               scratchpad=scratchpad)

print(f"✅ Game session created with {len(characters)} characters")
print(f"📝 Game log initialized (currently {len(game_session.game_log)} events)")
print(f"📝 Cache update interval {game_session.history_cache_manager.cache_update_interval} ")

🎭 Initializing game session...
📁 Loaded 1 cached basic metrics results
Pre-caching static content...
✅ Pre-cached system prompt and character context for Dungeon Master
✅ Pre-cached character context for Finlan
✅ Pre-cached character context for Masque
✅ Pre-cached character context for Cororin
✅ Game session created with 4 characters
📝 Game log initialized (currently 0 events)
📝 Cache update interval 10 


## Step 6: Run the Simulation

Now let's run the complete D&D simulation and watch our AI characters interact!

In [ ]:
# Run the simulation
print("🚀 Starting D&D simulation...")
print("\n" + "=" * 60)

# Execute the scenario 
game_session.run_scenario(initial_scenario=campaign_params['initial_scenario'],
                          turn_sequence=turn_sequence,
                          include_player_personalities=include_player_personalities,
                          print_cache=True)

🚀 Starting D&D simulation...

=== SYSTEM PROMPT ===
You are participating in a Dungeons & Dragons play-by-post forum game simulation.  GAME CONTEXT: - This is a turn-based roleplaying game where players control fantasy
characters - Each player posts actions and dialogue for their character in response to game situations - The Dungeon Master (DM) describes scenarios, environments, and
NPC interactions - Players should respond in character, matching typical play-by-post D&D forum style - Responses should include both actions and dialogue as appropriate
for the situation  CRITICAL TURN RESTRICTIONS:   - You are generating EXACTLY ONE turn for your assigned role (either a player character OR the Dungeon Master) - Do NOT
generate responses for other characters or roles  RESPONSE GUIDELINES: - Stay true to your character's personality, abilities, and background - Consider the current
situation and respond appropriately - Match the posting style and tone of play-by-post D&D forums - Include b

## Export Results

Save the simulation results for further analysis.

In [ ]:
# Export simulation results

# Save to file
if include_player_personalities:
    player_string = 'players'
else:
    player_string = 'no_players'

if scratchpad:
    scratch_string = 'scratch'
else:
    scratch_string = 'no_scratch'

if model[0:6] == 'gemini':
    model_str = model[7:]
else:
    model_str = model

output_file = f"llm_campaign_{campaign_name}_{model_str}_{player_string}_{scratch_string}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
output_path = os.path.join(os.getcwd(), '..', 'data', 'llm-games', output_file)
with open(output_path, 'w') as f:
    json.dump(game_session.game_log, f, indent=2)

print(f"💾 Results saved to: {output_path}")
print(f"📁 File size: {os.path.getsize(output_path)} bytes")

💾 Results saved to: /Users/annie/Code/Repositories/dnd-dynamics/tutorials/../data/llm-games/llm_campaign_20049-age-of-death_gemini-1.5-pro_no_players_scratch_20250818_130018.json
📁 File size: 95873 bytes
